> This NB contains all the code for preprocessing and saving the datasets. Originally used in June24, copied here for reference

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import time


In [ ]:
def manual_train_test_split(df_t, metadata_cols_df, save_file_name, save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets', user_holdout=True, gesture_holdout=False, held_out_user_pids=['P103','P109','P114','P124','P128','P005','P010'], held_out_test_gestures=['move', 'zoom-out', 'duplicate']):

    save_lst = []
    save_name_lst = []

    # Combine the data and metadata dfs again:
    # Ensure both DataFrames have the same index
    df_t.reset_index(drop=True, inplace=True)
    metadata_cols_df.reset_index(drop=True, inplace=True)
    # Concatenate the DataFrames
    metadata_PCA_df = pd.concat([metadata_cols_df, df_t], axis=1)
    save_lst.append(metadata_PCA_df)
    save_name_lst.append("full_dimreduc_df.pkl")

    if user_holdout:
        test_users_df = metadata_PCA_df[metadata_PCA_df['Participant'].isin(held_out_user_pids)]
        # Merge the DataFrames with an indicator
        merged_df = metadata_PCA_df.merge(test_users_df, how='left', indicator=True)
        # Filter out the rows that are in both DataFrames
        training_users_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
        save_lst.extend([test_users_df, training_users_df])
        #save_name_lst.extend([save_file_name+'_test_users_df.pkl', save_file_name+'_training_users_df.pkl'])
        save_name_lst.extend(['test_users_df.pkl', 'training_users_df.pkl'])
    if gesture_holdout:
        test_gestures_df = metadata_PCA_df[(metadata_PCA_df['Participant'].isin(held_out_user_pids)) & (data_df['Gesture_ID'].isin(held_out_gestures))]
        # Merge the DataFrames with an indicator
        merged_df = metadata_PCA_df.merge(test_gestures_df, how='left', indicator=True)
        # Filter out the rows that are in both DataFrames
        training_gestures_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')
        save_lst.extend([test_gestures_df, training_gestures_df])
        #save_name_lst.extend([save_file_name+'_test_gestures_df.pkl', save_file_name+'_training_gestures_df.pkl'])
        save_name_lst.extend(['test_gestures_df.pkl', 'training_gestures_df.pkl'])
        
    if save_bool:
        for idx, ele in enumerate(save_lst):
            ele.to_pickle(save_path + '\\' + save_file_name + '\\' + save_name_lst[idx])

    return save_lst, save_name_lst

In [ ]:
def preprocess_df_by_gesture(data_df, preprocessing_approach, biosignal_switch_ix_lst=[72], trial_length=64, zero_threshold=1e-7):
    '''
    Assumes there is no metadata cols (eg only pass in the 2D dataframe of sensor data
    Set biosignal_switch_ix_lst to an empty list if only working with one biosignal
    '''
    if len(biosignal_switch_ix_lst)>1:
        raise ValueError("Only one biosignal index switch is supported for now")
    
    num_trials = data_df.shape[0] // trial_length
    preprocessed_data_lst = []

    data_df = data_df.applymap(lambda x: 0.0 if x < zero_threshold else x)
    
    for trial_idx in range(num_trials):
        start_idx = trial_idx * trial_length
        end_idx = start_idx + trial_length
        trial_data = data_df.iloc[start_idx:end_idx]

        if trial_data.isna().any().any():
            print(f"Warning: NaNs detected in trial {trial_idx + 1} before preprocessing")

        if preprocessing_approach.upper() == 'MEANSUBTRACTION':
            # This already is applied on a per-column basis
            trial_data_preprocessed = trial_data - trial_data.mean()
        elif preprocessing_approach.upper() == 'MINMAXSCALER':
            scaler = MinMaxScaler()
            if len(biosignal_switch_ix_lst)==0:
                trial_data_preprocessed = scaler.fit_transform(trial_data)
            else:
                for biosignal_idx in biosignal_switch_ix_lst:
                    biosignal_block1 = trial_data.iloc[:, :biosignal_idx]
                    biosignal_block2 = trial_data.iloc[:, biosignal_idx:]
                    biosignal_block1_preprocessed = pd.DataFrame(scaler.fit_transform(biosignal_block1))
                    biosignal_block2_preprocessed = pd.DataFrame(scaler.fit_transform(biosignal_block2))
                    trial_data_preprocessed = pd.concat([biosignal_block1_preprocessed, biosignal_block2_preprocessed], axis=1)
        elif preprocessing_approach.upper() == 'STANDARDSCALER':
            scaler = StandardScaler()
            if len(biosignal_switch_ix_lst)==0:
                trial_data_preprocessed = scaler.fit_transform(trial_data)
            else:
                for biosignal_idx in biosignal_switch_ix_lst:
                    biosignal_block1 = trial_data.iloc[:, :biosignal_idx]
                    biosignal_block2 = trial_data.iloc[:, biosignal_idx:]
                    biosignal_block1_preprocessed = pd.DataFrame(scaler.fit_transform(biosignal_block1))
                    biosignal_block2_preprocessed = pd.DataFrame(scaler.fit_transform(biosignal_block2))
                    trial_data_preprocessed = pd.concat([biosignal_block1_preprocessed, biosignal_block2_preprocessed], axis=1)
            trial_data_preprocessed = scaler.fit_transform(trial_data)
        elif preprocessing_approach.upper() == '$B':
            ## This already is applied on a per-column basis
            #trial_data_centered = trial_data - trial_data.mean()
            trial_data_centered = trial_data
            if len(biosignal_switch_ix_lst)==0:
                gesture_std = np.std(trial_data_centered.values.flatten())
                if gesture_std != 0:
                    trial_data_preprocessed = pd.DataFrame(trial_data_centered / gesture_std)
                else:
                    print("Gesture STD is equal to 0!")
                    trial_data_preprocessed = pd.DataFrame(trial_data_centered)
            else:
                for biosignal_idx in biosignal_switch_ix_lst:
                    # IMU
                    biosignal_block1 = trial_data_centered.iloc[:, :biosignal_idx]
                    # EMG
                    biosignal_block2 = trial_data_centered.iloc[:, biosignal_idx:]

                    gesture_std1 = np.std(biosignal_block1.values.flatten())
                    if gesture_std1 != 0:
                        biosignal_block1_preprocessed = pd.DataFrame(biosignal_block1 / gesture_std1)
                    else:
                        print("Gesture STD is equal to 0!")
                        biosignal_block1_preprocessed = pd.DataFrame(biosignal_block1)
        
                    gesture_std2 = np.std(biosignal_block2.values.flatten())
                    if gesture_std2 != 0:
                        biosignal_block2_preprocessed = pd.DataFrame(biosignal_block2 / gesture_std2)
                    else:
                        print("Gesture STD is equal to 0!")
                        biosignal_block2_preprocessed = pd.DataFrame(biosignal_block2)

                    trial_data_preprocessed = pd.concat([biosignal_block1_preprocessed, biosignal_block2_preprocessed], axis=1)

            trial_data_centered = (trial_data - trial_data.mean()).applymap(lambda x: 0.0 if x < zero_threshold else x)

        trial_data_preprocessed_df = pd.DataFrame(trial_data_preprocessed, columns=data_df.columns)
        if trial_data_preprocessed_df.isna().any().any():
            print(f"Warning: NaNs detected in trial {trial_idx + 1} after preprocessing")
        preprocessed_data_lst.append(trial_data_preprocessed_df)
    
    preprocessed_df = pd.concat(preprocessed_data_lst, ignore_index=True)
    return preprocessed_df


def interpolate_df(df, num_rows=64, columns_to_exclude=None):
    """
    Interpolates the dataframe to have a specified number of rows.
    Excludes specified columns from interpolation.
    """
    if columns_to_exclude is None:
        columns_to_exclude = []
    
    # Separate the columns to exclude
    excluded_columns_df = df[columns_to_exclude]
    subset_df = df.drop(columns=columns_to_exclude)
    
    # Interpolate the remaining columns
    x_old = np.linspace(0, 1, num=len(subset_df))
    x_new = np.linspace(0, 1, num=num_rows)
    interpolated_df = pd.DataFrame()

    for column in subset_df.columns:
        y_old = subset_df[column].values
        y_new = np.interp(x_new, x_old, y_old)
        interpolated_df[column] = y_new

    # Add the excluded columns back
    for col in columns_to_exclude:
        interpolated_df[col] = excluded_columns_df[col].iloc[0]

    # Reordering columns to match original DataFrame
    interpolated_df = interpolated_df[columns_to_exclude + list(subset_df.columns)]
    
    return interpolated_df


def interpolate_dataframe(df, num_rows=64):
    '''Old version from Ben's code (/Momona?), assumes no meta data'''
    # Create a new index array with num_rows evenly spaced values
    new_index = np.linspace(0, len(df) - 1, num_rows)
    interpolated_df = pd.DataFrame(columns=df.columns, index=new_index)
    # Interpolate each column of the DataFrame using the new index
    for column in df.columns:
        interpolated_df[column] = np.interp(new_index, np.arange(len(df)), df[column])

    interpolated_df.index = range(num_rows)
    
    return interpolated_df

## Loading in the data

In [2]:
## Pickle is theoretically faster for Python...

print("Loading")
start_time = time.time()
#data_df = pd.read_pickle('C:\\Users\\kdmen\\Desktop\\Research\\Data\\$M\\filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl')
data_df = pd.read_pickle('D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets\\metadata_IMU_EMG_allgestures_allusers.pkl')
end_time = time.time()
print(f"Completed in {end_time - start_time}s")

Loading
Completed in 0.10723662376403809s


In [3]:
print(data_df.shape)
data_df.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,0.341797,-0.939941,0.000977,-0.007450,-0.192625,0.005321,-0.380859,...,0.000002,0.000002,0.000003,0.000020,0.000004,0.000004,0.000002,0.000009,0.000001,0.000002
1,P102,pan,1,0.336178,-0.963185,0.003898,0.009595,-0.190446,-0.026116,-0.394547,...,0.000003,0.000003,0.000003,0.000014,0.000007,0.000007,0.000002,0.000017,0.000001,0.000002
2,P102,pan,1,0.353539,-0.963704,0.011711,0.095966,-0.205480,-0.155563,-0.398406,...,0.000003,0.000003,0.000004,0.000007,0.000004,0.000005,0.000003,0.000020,0.000003,0.000002
3,P102,pan,1,0.352841,-0.950288,0.011509,0.058836,-0.184871,-0.083567,-0.389230,...,0.000003,0.000003,0.000006,0.000005,0.000004,0.000003,0.000004,0.000015,0.000003,0.000003
4,P102,pan,1,0.372621,-0.991273,0.029847,0.293946,-0.178756,-0.281361,-0.396043,...,0.000003,0.000002,0.000008,0.000003,0.000007,0.000022,0.000004,0.000017,0.000002,0.000003


In [4]:
metadata_cols = ['Participant', 'Gesture_ID', 'Gesture_Num']
metadata_cols_df = data_df[metadata_cols]
X = data_df.drop(metadata_cols, axis=1)

In [5]:
print(metadata_cols_df.shape)
metadata_cols_df.head()

(204800, 3)


,Participant,Gesture_ID,Gesture_Num
0,P102,pan,1
1,P102,pan,1
2,P102,pan,1
3,P102,pan,1
4,P102,pan,1


## Mean subtract each column, standardize each gesture

In [6]:
imu_split_df = X.iloc[:, :72]
emg_split_df = X.iloc[:, 72:]

ppd_emg_df = preprocess_df_by_gesture(emg_split_df, '$B', biosignal_switch_ix_lst=[])
ppd_imu_df = preprocess_df_by_gesture(imu_split_df, '$B', biosignal_switch_ix_lst=[])
X_ppd = pd.concat([metadata_cols_df, ppd_imu_df, ppd_emg_df], axis=1)

ppd_imu_df = pd.concat([metadata_cols_df, ppd_imu_df], axis=1)
ppd_emg_df = pd.concat([metadata_cols_df, ppd_emg_df], axis=1)

print(X_ppd.shape)
X_ppd.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [7]:
print(ppd_imu_df.shape)
ppd_imu_df.head()

(204800, 75)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,IMU13_az,IMU13_vx,IMU13_vy,IMU13_vz,IMU15_ax,IMU15_ay,IMU15_az,IMU15_vx,IMU15_vy,IMU15_vz
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.047646,-0.045230,0.104149,-0.046899,-0.027093,0.181512,0.003823,0.002525,0.055640,-0.009292
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.008147,-0.074827,0.006427,-0.049691,0.007038,0.184160,-0.014212,0.009704,0.034081,-0.026250
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.009861,-0.125477,0.203083,-0.095080,-0.077138,0.186451,0.034482,-0.061853,0.137769,-0.009996
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,0.008394,-0.174411,-0.027033,-0.137434,-0.025918,0.193092,0.016815,-0.050325,-0.040597,-0.040180
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,0.003086,-0.259050,0.118230,-0.111992,-0.081781,0.165432,0.029207,-0.197250,-0.006715,-0.030144


In [8]:
print(ppd_emg_df.shape)
ppd_emg_df.head()

(204800, 19)


,Participant,Gesture_ID,Gesture_Num,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.362743,-0.801651,-0.383077,-0.195299,-0.203047,-0.464472,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.351553,-0.775334,-0.382545,-0.154773,-0.131977,-0.295204,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.380825,-0.762588,-0.398388,-0.085411,0.017528,-0.205675,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.366795,-0.765464,-0.374423,-0.073225,0.183172,0.009277,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.245578,-0.761283,-0.303976,-0.081947,0.224996,0.103319,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


## Run train test splits and save

In [19]:
both_ppd_return_lst, save_name_lst = manual_train_test_split(X_ppd.iloc[:, 3:], metadata_cols_df, 'Both_PPD', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets', user_holdout=True, gesture_holdout=False)


In [20]:
emg_ppd_return_lst = manual_train_test_split(ppd_emg_df.iloc[:, 3:], metadata_cols_df, 'EMG_PPD', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets', user_holdout=True, gesture_holdout=False)


## Now do a train test split that only includes the participants with disabilities

In [26]:
pIDs_impaired = ['P102','P103','P104','P105','P106','P107','P108','P109','P110','P111',
       'P112','P114','P115','P116','P118','P119','P121','P122','P123','P124','P125',
       'P126','P127','P128', 'P131', 'P132']
# remove participants P001 and P003 because they dont have duplicate or open gestures
pIDs_unimpaired = ['P004','P005','P006','P008','P010','P011']

pIDs_both = pIDs_impaired + pIDs_unimpaired

In [29]:
print(X_ppd.shape)
X_ppd.head()

(204800, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [28]:
dis_ppd_df = X_ppd[X_ppd['Participant'].isin(pIDs_impaired)]
print(dis_ppd_df.shape)
dis_ppd_df.head()

(166400, 91)


,Participant,Gesture_ID,Gesture_Num,IMU1_ax,IMU1_ay,IMU1_az,IMU1_vx,IMU1_vy,IMU1_vz,IMU2_ax,...,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,P102,pan,1,-0.551109,-0.738972,-0.985439,0.181924,0.059616,0.087024,1.055804,...,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,P102,pan,1,-0.571115,-0.821726,-0.975036,0.242607,0.067375,-0.024900,1.007074,...,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,P102,pan,1,-0.509305,-0.823575,-0.947221,0.550111,0.013848,-0.485765,0.993332,...,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,P102,pan,1,-0.511788,-0.775810,-0.947939,0.417919,0.087222,-0.229441,1.026003,...,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,P102,pan,1,-0.441369,-0.921726,-0.882652,1.254970,0.108993,-0.933639,1.001748,...,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [30]:
dis_ppd_df['Participant'].unique()

array(['P102', 'P103', 'P104', 'P105', 'P106', 'P107', 'P108', 'P109',
       'P110', 'P111', 'P112', 'P114', 'P115', 'P116', 'P118', 'P119',
       'P121', 'P122', 'P123', 'P124', 'P125', 'P126', 'P127', 'P128',
       'P131', 'P132'], dtype=object)

In [34]:
both_ppd_return_lst, save_name_lst = manual_train_test_split(dis_ppd_df.iloc[:, 3:], metadata_cols_df, 'Both_PPD', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets\\with_disabilities', held_out_user_pids=['P103','P109','P114','P124','P128'], user_holdout=True, gesture_holdout=False)


In [33]:
dis_emg_ppd_df = dis_ppd_df.iloc[:, 75:]
dis_emg_ppd_df.head()

,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12,EMG13,EMG14,EMG15,EMG16
0,-0.362743,-0.801651,-0.383077,-0.195299,-0.203047,-0.464472,-0.276292,-0.026736,-0.873870,-1.036152,-0.580930,-0.719494,-0.502255,-1.750091,-0.127847,-0.094192
1,-0.351553,-0.775334,-0.382545,-0.154773,-0.131977,-0.295204,-0.125822,0.089679,-0.816215,-2.082635,-0.006283,-0.139439,-0.367764,-0.208084,-0.111811,-0.039009
2,-0.380825,-0.762588,-0.398388,-0.085411,0.017528,-0.205675,-0.068451,0.117076,-0.668221,-3.403064,-0.526030,-0.478294,-0.300443,0.203266,0.113300,0.004728
3,-0.366795,-0.765464,-0.374423,-0.073225,0.183172,0.009277,-0.058907,0.080977,-0.424416,-3.709413,-0.570894,-0.775155,-0.144710,-0.619539,0.146499,0.199975
4,-0.245578,-0.761283,-0.303976,-0.081947,0.224996,0.103319,-0.003929,0.041526,-0.016530,-4.075150,-0.127710,2.682791,-0.141750,-0.208404,-0.035642,0.172662


In [35]:
emg_ppd_return_lst = manual_train_test_split(dis_emg_ppd_df, metadata_cols_df, 'EMG_PPD', save_bool=True, save_path='D:\\Kai_MetaGestureClustering_24\\saved_datasets\\filtered_datasets\\with_disabilities', held_out_user_pids=['P103','P109','P114','P124','P128'], user_holdout=True, gesture_holdout=False)
